### LLAMA model 
- Classify the seriousness of the subredits
- Classify the reddit post  


In [ ]:
%pip install torch torchvision torchaudio
%pip install transformers
# Make sure you have Hugging Face account and token for accessing LLaMA models
%pip install huggingface_hub

In [3]:
from transformers import AutoModel, AutoTokenizer

# Specify the model repository
model_name = "meta-llama/Llama-3.2-1B"

# Download the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, local_files_only=False)

# Save locally if needed
model.save_pretrained("./Llama-3.2-1B")
tokenizer.save_pretrained("./Llama-3.2-1B")

('./Llama-3.2-1B/tokenizer_config.json',
 './Llama-3.2-1B/special_tokens_map.json',
 './Llama-3.2-1B/tokenizer.json')

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the tokenizer and model (already saved locally)
tokenizer = AutoTokenizer.from_pretrained("./Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("./Llama-3.2-1B")

# Set pad token ID to eos token ID if pad token ID is not set
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Test the model
input_text = (
    "Answer the following question with a single word: yes or no.\n"
    "Question: Is Paris the Capital of Spain?"
)
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Create attention mask
attention_mask = input_ids.ne(tokenizer.pad_token_id).long()

# Generate a response
output = model.generate(input_ids, attention_mask=attention_mask, max_length=50, pad_token_id=tokenizer.eos_token_id)
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)

Answer the following question with a single word: yes or no.
Question: Is Paris the Capital of Spain?


#### Connect to Mongdb

In [1]:
from dotenv import load_dotenv
from pymongo import MongoClient

# Load environment variables from .env file
load_dotenv()
mongo='127.0.0.1'

try:
    # Connect to MongoDB
    myclient = MongoClient(
                        "mongodb://"+mongo+":27017/",  
                        username='admin',
                        password='admin') #Mongo URI format
    db=myclient['reddit']
    print("Connected to MongoDB successfully!")
except Exception as e:
    print("An error occurred while connecting to MongoDB:", e)

Connected to MongoDB successfully!


In [2]:
#import
import pandas as pd
posts_registered=[]
query=db.reddit_posts.find({},{'_id':0})
for el in query:
    posts_registered.append(el)
posts_registered_df=pd.DataFrame(posts_registered)
posts_registered_df.head()

,id,title,author,score,num_comments,upvote_ratio,url,subreddit,created_at,self_text,searchQuery
0,1et3kj0,Diagnosed with Inattentive ADHD at 31. Explain...,amadnomad,392,107,0.98,https://www.reddit.com/r/ADHD/comments/1et3kj0...,ADHD,1.723749e+09,Please go out and get tested if you are still ...,adhd
1,1gk5ftv,Children with higher IQ scores were diagnosed ...,Pretend_Voice_3140,4499,466,0.99,https://www.reddit.com/r/ADHD/comments/1gk5ftv...,ADHD,1.730809e+09,A study was published in the [British Journal ...,adhd
2,1f0k9en,Reminder: If you made it to adulthood with lat...,Hipster-Deuxbag,7372,535,0.99,https://www.reddit.com/r/ADHD/comments/1f0k9en...,ADHD,1.724547e+09,"We all know the statistics: 20,000 behavioral ...",adhd
3,1g7tbb2,What is the most adhd thing you have ever adhd’ed,FamiliarRadio9275,2102,1022,0.99,https://www.reddit.com/r/ADHD/comments/1g7tbb2...,ADHD,1.729409e+09,I laugh so hard when I look back at this memor...,adhd
4,1dy8uqs,I’m angry that no one recognized that I had AD...,thecynicalone26,3345,598,0.99,https://www.reddit.com/r/ADHD/comments/1dy8uqs...,ADHD,1.720445e+09,"I just got diagnosed, and I’m 39. My entire l...",adhd


In [3]:
print(posts_registered_df.iloc[0]['title'])
print(posts_registered_df.iloc[0]['self_text'])
title_to_test=posts_registered_df.iloc[0]['title']
text_to_test=posts_registered_df.iloc[0]['self_text']

Diagnosed with Inattentive ADHD at 31. Explains so many things from my childhood.
Please go out and get tested if you are still on the fence. I always assumed ADHD was only hyperactive. A lot of concerns about day dreaming, zoning out and inattentiveness came into play during my consult. I didn't even consider my lack of sleep being tied to ADHD. But now that I have a diagnoses, it explains quite a bit from my past. I wasn't just lazy and disorganized. 

  
Again, please go get tested if you suspect anything.


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the tokenizer and model (already saved locally)
tokenizer = AutoTokenizer.from_pretrained("./Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("./Llama-3.2-1B")

# Set pad token ID to eos token ID if pad token ID is not set
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Test the model
input_text = f"""Can you help me with this question? Please analyze the following and provide insights:
Title: {title_to_test}
Text: {text_to_test}

Provide an insightful and concise analysis below:
"""

input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Create attention mask
attention_mask = input_ids.ne(tokenizer.pad_token_id).long()

# Generate a response
output = model.generate(
    input_ids,
    attention_mask=attention_mask,
    max_length=200,  # Length of the response
    temperature=0.7,  # Balance between random and deterministic
    top_p=0.9,        # Focus on top 90% of likely tokens
    pad_token_id=tokenizer.eos_token_id
)
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)

/home/ysidhom/Documents/5IF/adhd_project/mental_health_disorders_analysis/.venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/ysidhom/Documents/5IF/adhd_project/mental_health_disorders_analysis/.venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Can you help me with this question? Please analyze the following and provide insights:
Title: Diagnosed with Inattentive ADHD at 31. Explains so many things from my childhood.
Text: Please go out and get tested if you are still on the fence. I always assumed ADHD was only hyperactive. A lot of concerns about day dreaming, zoning out and inattentiveness came into play during my consult. I didn't even consider my lack of sleep being tied to ADHD. But now that I have a diagnoses, it explains quite a bit from my past. I wasn't just lazy and disorganized. 

  
Again, please go get tested if you suspect anything.

Provide an insightful and concise analysis below:

